# Initialize the Python Notebook

Install the necessary Python packages found in requirements.txt and import the necessary Python libraries.

In [1]:
%%capture
!pip3 install -r requirements.txt

import nltk
import os
import shutil
import uuid
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from pyspark.sql import SparkSession, Window, functions as F
from pyspark.sql.types import (
    DateType,
    DecimalType,
    IntegerType,
    LongType,
    StructType,
    StructField,
    StringType,
    TimestampType,
)
from sumy.nlp.stemmers import Stemmer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lsa import LsaSummarizer
from sumy.utils import get_stop_words

Initialize the Python notebook environment.

In [2]:
# A sentence tokenizer to split text into sentences
nltk.download("punkt_tab")

# Initialize a Spark session
# spark = SparkSession.builder.appName("Stock Forecasting").getOrCreate()
spark = (
    SparkSession.builder.appName("Stock Forecasting")
    .config("spark.executor.memory", "4g")
    .config("spark.driver.memory", "2g")
    .config("spark.default.parallelism", "8")
    .getOrCreate()
)

# Stock symbol
STOCK_SYMBOL_UPPER = "MSFT"  # AA, AAPL, AMZN, MSFT, TSLA
STOCK_SYMBOL_LOWER = "msft"  # aa, aapl, amzn, msft, tsla

# File names
FILE_NAME_NEWS = f"news_{STOCK_SYMBOL_LOWER}.csv"
FILE_NAME_PRICE = f"price_{STOCK_SYMBOL_LOWER}.csv"
FILE_NAME_COMBINED = f"combined_{STOCK_SYMBOL_LOWER}.csv"

# File paths
FILE_PATH_EXTERNAL_NEWS = f"stock_news/external/external_{STOCK_SYMBOL_LOWER}.csv"
FILE_PATH_NASDAQ_NEWS = f"stock_news/nasdaq/nasdaq_{STOCK_SYMBOL_LOWER}.csv"
FILE_PATH_MERGED_NEWS = f"stock_news/merged/{FILE_NAME_NEWS}"
FILE_PATH_SUMMARIZED_NEWS = f"stock_news/summarized/{FILE_NAME_NEWS}"
FILE_PATH_SCORED_NEWS = f"stock_news/scored/{FILE_NAME_NEWS}"
FILE_PATH_DECAYED_NEWS = f"stock_news/decayed/{FILE_NAME_NEWS}"
FILE_PATH_SENTIMENT_SCORES = f"stock_news/scored/sentiments_{STOCK_SYMBOL_LOWER}.csv"
FILE_PATH_FULL_HISTORY_PRICE = f"stock_price/full_history/{STOCK_SYMBOL_UPPER}.csv"
FILE_PATH_PREPROCESSED_PRICE = f"stock_price/preprocessed/{FILE_NAME_PRICE}"

# Used by the Tokenizer
LANGUAGE = "english"  # Set the language

# Used by the LSA Summarizer
SENTENCES_COUNT = 3  # Set the max number of sentences in a summary

# Used by the LLM
# MODEL = "llama3.2:1b-instruct-fp16"
MODEL = "llama3.2:3b-instruct-fp16"  # Set the large-language model
BATCH_SIZE = 5  # Set the max number of sentences in a batch to be fed to the LLM
TEMPERATURE = 0.0  # Set the temperature to 0.0 for deterministic output
MAX_OUTPUT_TOKENS = 14  # Set the maximum number of output tokens

# Used by LLM for sentiment scoring
MIN_VALUE = 1  # The minimum value of the sentiment score
BASE_VALUE = 3  # The midpoint between 1 and 5 of the sentiment score
MAX_VALUE = 5  # The maximum value of the sentiment score

# Used by exponential decay algorithm
DECAY_RATE = 0.5  # Determines how quickly the sentiment decays over time

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/24 20:19:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/24 20:19:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Preprocess the Price Dataset

Ingest the price dataset.

In [3]:
# Define the schema
price_schema = StructType(
    [
        StructField("date", DateType(), True),
        StructField("open", DecimalType(24, 16), True),
        StructField("high", DecimalType(24, 16), True),
        StructField("low", DecimalType(24, 16), True),
        StructField("close", DecimalType(24, 16), True),
        StructField("adj close", DecimalType(24, 16), True),
        StructField("volume", LongType(), True),
    ]
)

# Read price data from CSV files to Spark dataframes
df_price = spark.read.csv(
    FILE_PATH_FULL_HISTORY_PRICE, header=True, schema=price_schema
)

# Store in memory
df_price.persist()

# Verify
print(f"Row count for df_price: {df_price.count()}")
df_price.show(5, truncate=True)

Row count for df_price: 6700
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|      date|                open|                high|                 low|               close|           adj close|  volume|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|2023-12-28|153.7200012207031200|154.0800018310547000|152.9499969482422000|153.3800048828125000|153.3800048828125000|27031200|
|2023-12-27|153.5599975585937500|154.7799987792968800|153.1199951171875000|153.3399963378906200|153.3399963378906200|31434700|
|2023-12-26|153.5599975585937500|153.9799957275390600|153.0299987792968800|153.4100036621093800|153.4100036621093800|25067200|
|2023-12-22|153.7700042724609400|154.3500061035156200|152.7100067138672000|153.4199981689453000|153.4199981689453000|29480100|
|2023-12-21|153.3000030517578000|153.9700012207031200|152.1000061035156200|153.839

Preprocess the price dataset.

In [4]:
# Rename the headers
df_price = (
    df_price.withColumnRenamed("date", "Date")
    .withColumnRenamed("open", "Open")
    .withColumnRenamed("high", "High")
    .withColumnRenamed("low", "Low")
    .withColumnRenamed("close", "Close")
    .withColumnRenamed("adj close", "Adj_close")
    .withColumnRenamed("volume", "Volume")
    .orderBy("Date")
)

# Verify
print(f"Row count for df_price: {df_price.count()}")
df_price.show(5, truncate=True)

Row count for df_price: 6700
+----------+------------------+------------------+------------------+------------------+------------------+--------+
|      Date|              Open|              High|               Low|             Close|         Adj_close|  Volume|
+----------+------------------+------------------+------------------+------------------+------------------+--------+
|1997-05-15|2.4375000000000000|2.5000000000000000|1.9270833730697632|1.9583333730697632|1.9583333730697632|72156000|
|1997-05-16|1.9687500000000000|1.9791666269302368|1.7083333730697632|1.7291666269302368|1.7291666269302368|14700000|
|1997-05-19|1.7604166269302368|1.7708333730697632|1.6250000000000000|1.7083333730697632|1.7083333730697632| 6106800|
|1997-05-20|1.7291666269302368|1.7500000000000000|1.6354166269302368|1.6354166269302368|1.6354166269302368| 5467200|
|1997-05-21|1.6354166269302368|1.6458333730697632|1.3750000000000000|1.4270833730697632|1.4270833730697632|18853200|
+----------+------------------+----

Save a copy of the dataset.

In [5]:
# Specify directories
FOLDER_PRICE = "stock_price/preprocessed"
TEMP_FOLDER_PRICE = f"stock_price/preprocessed/price_{STOCK_SYMBOL_LOWER}"

# Write to a single CSV file
df_price.coalesce(1).write.csv(
    TEMP_FOLDER_PRICE, sep=",", header=True, mode="overwrite"
)

# Move the part file to the desired filename
for filename in os.listdir(TEMP_FOLDER_PRICE):
    if filename.startswith("part-"):
        shutil.move(
            os.path.join(TEMP_FOLDER_PRICE, filename),
            os.path.join(FOLDER_PRICE, FILE_NAME_PRICE),
        )

# Remove the temporary directory
shutil.rmtree(TEMP_FOLDER_PRICE)

# Uncache the dataframe in a non-blocking operation to free up memory.
df_price.unpersist(blocking=False)

# Delete the dataframe
del df_price

# Preprocess the News Datasets

Read in the external news and Nasdaq news datasets from CSV files to Spark dataframes.

In [6]:
# Define the schemas
external_schema = StructType(
    [
        StructField("Date", TimestampType(), True),
        StructField("Article_title", StringType(), True),
        StructField("Stock_symbol", StringType(), True),
        StructField("Url", StringType(), True),
        StructField("Publisher", StringType(), True),
        StructField("Author", StringType(), True),
        StructField("Article", StringType(), True),
        StructField("Lsa_summary", StringType(), True),
        StructField("Luhn_summary", StringType(), True),
        StructField("Textrank_summary", StringType(), True),
        StructField("Lexrank_summary", StringType(), True),
    ]
)
nasdaq_schema = StructType(
    [
        StructField("Unnamed: 0", StringType(), True),
        StructField("Date", TimestampType(), True),
        StructField("Article_title", StringType(), True),
        StructField("Stock_symbol", StringType(), True),
        StructField("Url", StringType(), True),
        StructField("Publisher", StringType(), True),
        StructField("Author", StringType(), True),
        StructField("Article", StringType(), True),
        StructField("Lsa_summary", StringType(), True),
        StructField("Luhn_summary", StringType(), True),
        StructField("Textrank_summary", StringType(), True),
        StructField("Lexrank_summary", StringType(), True),
    ]
)

# Read news data from CSV files to Spark dataframes
df_external_news = spark.read.csv(
    FILE_PATH_EXTERNAL_NEWS, header=True, schema=external_schema
)
df_nasdaq_news = spark.read.csv(
    FILE_PATH_NASDAQ_NEWS, header=True, schema=nasdaq_schema
)

# Store in memory
df_external_news.persist()
df_nasdaq_news.persist()

# Verify the dataframes
print(f"Row count for df_external_news: {df_external_news.count()}")
df_external_news.show(5, truncate=True)

print(f"Row count for df_nasdaq_news: {df_nasdaq_news.count()}")
df_nasdaq_news.show(5, truncate=True)

Row count for df_external_news: 278
+-------------------+--------------------+------------+--------------------+-----------------+------+-------+-----------+------------+----------------+---------------+
|               Date|       Article_title|Stock_symbol|                 Url|        Publisher|Author|Article|Lsa_summary|Luhn_summary|Textrank_summary|Lexrank_summary|
+-------------------+--------------------+------------+--------------------+-----------------+------+-------+-----------+------------+----------------+---------------+
|2020-06-10 17:18:50|How Cannabis Comp...|        AMZN|https://www.benzi...|      Jaycee Tenn|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|
|2020-06-10 15:33:26|Tech Stocks And F...|        AMZN|https://www.benzi...|       JJ Kinahan|  NULL|   NULL|       NULL|        NULL|            NULL|           NULL|
|2020-06-10 11:53:47|Benzinga Pro's To...|        AMZN|https://www.benzi...|Benzinga Newsdesk|  NULL|   NULL|       NULL|   

Drop unused fields and merge the external and Nasdaq news datasets into one dataframe.

In [7]:
# Drop unused fields
df_external_news = df_external_news.drop(
    "Luhn_summary", "Textrank_summary", "Lexrank_summary"
)
df_nasdaq_news = df_nasdaq_news.drop(
    "Unnamed: 0", "Luhn_summary", "Textrank_summary", "Lexrank_summary"
)

# Merge two dataframes
df_news = df_nasdaq_news.unionByName(df_external_news)

# Store in memory
df_news.persist()

# Verify
# Expect count is 7419, where 2945 + 4474 = 7419
print(f"Row count for df_news: {df_news.count()}")
df_news.show(5)

Row count for df_news: 5331
+-------------------+--------------------+------------+--------------------+---------+------+--------------------+-----------+
|               Date|       Article_title|Stock_symbol|                 Url|Publisher|Author|             Article|Lsa_summary|
+-------------------+--------------------+------------+--------------------+---------+------+--------------------+-----------+
|2023-12-17 07:00:00|What Are the Best...|        AMZN|https://www.nasda...|     NULL|  NULL|It's nearly impos...|       NULL|
|2023-12-17 07:00:00|Amazon (AMZN) Sto...|        AMZN|https://www.nasda...|     NULL|  NULL|Amazon (AMZN) clo...|       NULL|
|2023-12-17 05:00:00|3 Top Ranked Stoc...|        AMZN|https://www.nasda...|     NULL|  NULL|It’s no secret th...|       NULL|
|2023-12-17 03:00:00|TotalEnergies (TT...|        AMZN|https://www.nasda...|     NULL|  NULL|TotalEnergies SE ...|       NULL|
|2023-12-17 02:00:00|US 'buy now, pay ...|        AMZN|https://www.nasda...|     NU

Preprocess the merged news dataset.

In [8]:
# Standardize the timestamps to UTC timezone. Example: Convert "2019-01-15 00:00:00 UTC" to "2019-01-15 00:00:00".
df_news = df_news.withColumn(
    "Date", F.to_utc_timestamp(F.to_timestamp("Date", "yyyy-MM-dd HH:mm:ss zzz"), "UTC")
).filter(F.col("Date").isNotNull())

# Add a "Summarized" field with all values set to 0.
# Add a "Sentiment_score" field with all values set to 0.
# Sort by Date field in descending order.
df_news = (
    df_news.withColumn("Summarized", F.lit(0))
    .withColumn("Sentiment_score", F.lit(0))
    .orderBy("Date", ascending=False)
)

# Add a unique identifier field
uuid_udf = F.udf(lambda: str(uuid.uuid4()), StringType())
df_news = df_news.withColumn("UUID", uuid_udf())

# Verify
print(f"Row count for df_news: {df_news.count()}")
df_news.show(5, truncate=True)

Row count for df_news: 5331


+-------------------+--------------------+------------+--------------------+---------+------+--------------------+-----------+----------+---------------+--------------------+
|               Date|       Article_title|Stock_symbol|                 Url|Publisher|Author|             Article|Lsa_summary|Summarized|Sentiment_score|                UUID|
+-------------------+--------------------+------------+--------------------+---------+------+--------------------+-----------+----------+---------------+--------------------+
|2023-12-17 07:00:00|What Are the Best...|        AMZN|https://www.nasda...|     NULL|  NULL|It's nearly impos...|       NULL|         0|              0|6c14c3f0-9987-46c...|
|2023-12-17 07:00:00|Amazon (AMZN) Sto...|        AMZN|https://www.nasda...|     NULL|  NULL|Amazon (AMZN) clo...|       NULL|         0|              0|3dc6b09e-3b19-482...|
|2023-12-17 05:00:00|3 Top Ranked Stoc...|        AMZN|https://www.nasda...|     NULL|  NULL|It’s no secret th...|       NULL

Save a copy of the preprocessed merged news dataset.

In [9]:
# Specify directories
FOLDER_MERGED_NEWS = "stock_news/merged"
TEMP_FOLDER_MERGED_NEWS = f"stock_news/merged/news_{STOCK_SYMBOL_LOWER}"

# Write to a single CSV file
df_news.coalesce(1).write.csv(
    TEMP_FOLDER_MERGED_NEWS, sep=",", header=True, mode="overwrite"
)

# Move the part file to the desired filename
for filename in os.listdir(TEMP_FOLDER_MERGED_NEWS):
    if filename.startswith("part-"):
        shutil.move(
            os.path.join(TEMP_FOLDER_MERGED_NEWS, filename),
            os.path.join(FOLDER_MERGED_NEWS, FILE_NAME_NEWS),
        )

# Remove the temporary directory
shutil.rmtree(TEMP_FOLDER_MERGED_NEWS)

# Uncache the dataframe in a non-blocking operation to free up memory.
df_external_news.unpersist(blocking=False)
df_nasdaq_news.unpersist(blocking=False)
df_news.unpersist(blocking=False)

# Delete vairables
del external_schema, nasdaq_schema, df_external_news, df_nasdaq_news, df_news, uuid_udf

# Summarize the News Dataset

Retrieve the merged news dataset.

In [10]:
# Define the schemas
merged_news_schema = StructType(
    [
        StructField("Date", TimestampType(), True),
        StructField("Article_title", StringType(), True),
        StructField("Stock_symbol", StringType(), True),
        StructField("Url", StringType(), True),
        StructField("Publisher", StringType(), True),
        StructField("Author", StringType(), True),
        StructField("Article", StringType(), True),
        StructField("Lsa_summary", StringType(), True),
        StructField("Summarized", IntegerType(), True),
        StructField("Sentiment_score", IntegerType(), True),
        StructField("UUID", StringType(), True),
    ]
)

# Read the CSV file to Spark dataframe
df_merged_news = spark.read.csv(
    FILE_PATH_MERGED_NEWS, header=True, schema=merged_news_schema
)

# Store in memory
df_merged_news.persist()

# Verify
print(f"Row count for df_merged_news: {df_merged_news.count()}")
df_merged_news.show(5, truncate=True)

Row count for df_merged_news: 5331
+-------------------+--------------------+------------+--------------------+---------+------+--------------------+-----------+----------+---------------+--------------------+
|               Date|       Article_title|Stock_symbol|                 Url|Publisher|Author|             Article|Lsa_summary|Summarized|Sentiment_score|                UUID|
+-------------------+--------------------+------------+--------------------+---------+------+--------------------+-----------+----------+---------------+--------------------+
|2023-12-17 07:00:00|What Are the Best...|        AMZN|https://www.nasda...|     NULL|  NULL|It's nearly impos...|       NULL|         0|              0|003d7667-7beb-452...|
|2023-12-17 07:00:00|Amazon (AMZN) Sto...|        AMZN|https://www.nasda...|     NULL|  NULL|Amazon (AMZN) clo...|       NULL|         0|              0|2ffd76dd-00ef-44e...|
|2023-12-17 05:00:00|3 Top Ranked Stoc...|        AMZN|https://www.nasda...|     NULL|  NU

Summarize the news texts using a LSA Summarizer.

In [11]:
"""
Description:
Takes the article text as input, parses it using PlaintextParser, and summarizes it using LsaSummarizer.

Parameters:
text (string): The news article.

Returns:
string: The summarized text.
"""


def summarize_article(text):
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))

    # Stemming reduces words to their root form for the summarizer to identify similar concepts expressed with
    # different word forms.
    stemmer = Stemmer(LANGUAGE)

    # Initializes the summarizer with the stemmer
    summarizer = LsaSummarizer(stemmer)

    # Removes stop word to eliminate common non-keyword words.
    summarizer.stop_words = get_stop_words(LANGUAGE)

    # Generates the summarized text
    summary = summarizer(parser.document, SENTENCES_COUNT)
    return " ".join([str(sentence) for sentence in summary])


# A user-designed function to wrap the summarize_article function to be used in Spark.
summarize_udf = F.udf(summarize_article, StringType())

# Concats both fields with a period and space as the separator.
# Execute the summarize_udf function
df_merged_news = df_merged_news.withColumn(
    "Text", F.concat_ws(". ", "Article_title", "Article")
).withColumn("Lsa_summary", summarize_udf("Text"))

# Updated summarized indicator
df_merged_news = df_merged_news.withColumn(
    "Summarized",
    F.when(
        F.col("Lsa_summary").isNotNull() & (F.col("Lsa_summary") != ""), F.lit(1)
    ).otherwise(0),
)

# Verify number of news with no summary
# print(
#     f"Number of news with no summary: {df_merged_news.filter(col("Summarized") == 0).count()}"
# )

# Remove rows without a summary
df_merged_news = df_merged_news.filter((F.col("Summarized") == 1))

Verify the results of the summarization.

In [12]:
# Verify
print(f"Row count for df_merged_news: {df_merged_news.count()}")

df_merged_news.show(5, truncate=True)

df_merged_news.select(
    "Date",
    "Summarized",
    "Text",
    "Lsa_summary",
).show(10, truncate=False)

Row count for df_merged_news: 5331


24/10/24 14:09:12 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 47 (TID 43): Attempting to kill Python Worker


+-------------------+--------------------+------------+--------------------+---------+------+--------------------+--------------------+----------+---------------+--------------------+--------------------+
|               Date|       Article_title|Stock_symbol|                 Url|Publisher|Author|             Article|         Lsa_summary|Summarized|Sentiment_score|                UUID|                Text|
+-------------------+--------------------+------------+--------------------+---------+------+--------------------+--------------------+----------+---------------+--------------------+--------------------+
|2023-12-17 07:00:00|What Are the Best...|        AMZN|https://www.nasda...|     NULL|  NULL|It's nearly impos...|1 Wall Street Ana...|         1|              0|003d7667-7beb-452...|What Are the Best...|
|2023-12-17 07:00:00|Amazon (AMZN) Sto...|        AMZN|https://www.nasda...|     NULL|  NULL|Amazon (AMZN) clo...|Amazon (AMZN) Sto...|         1|              0|2ffd76dd-00ef-44e.

+-------------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Date               |Summarized|Text                                                                                                                       

24/10/24 14:09:22 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 48 (TID 44): Attempting to kill Python Worker


Save a copy of the summarized news dataset.

In [13]:
# Specify the directories
FOLDER_SUMMARIZED_NEWS = "stock_news/summarized"
TEMP_FOLDER_SUMMARIZED_NEWS = "stock_news/summarized/news_{STOCK_SYMBOL_LOWER}"

# Write to a single CSV file
df_merged_news.coalesce(1).write.csv(
    TEMP_FOLDER_SUMMARIZED_NEWS, sep=",", header=True, mode="overwrite"
)

# Move the part file to the desired filename
for filename in os.listdir(TEMP_FOLDER_SUMMARIZED_NEWS):
    if filename.startswith("part-"):
        shutil.move(
            os.path.join(TEMP_FOLDER_SUMMARIZED_NEWS, filename),
            os.path.join(FOLDER_SUMMARIZED_NEWS, FILE_NAME_NEWS),
        )

# Remove the temporary directory
shutil.rmtree(TEMP_FOLDER_SUMMARIZED_NEWS)

# Uncache the dataframe in a non-blocking operation to free up memory.
df_merged_news.unpersist(blocking=False)

# Delete variables
del merged_news_schema, df_merged_news, summarize_udf

# Calculate the Sentiment Score of the News

Retrieve the summarized news dataset.

In [13]:
# Define the schemas
summarized_news_schema = StructType(
    [
        StructField("Date", TimestampType(), True),
        StructField("Article_title", StringType(), True),
        StructField("Stock_symbol", StringType(), True),
        StructField("Url", StringType(), True),
        StructField("Publisher", StringType(), True),
        StructField("Author", StringType(), True),
        StructField("Article", StringType(), True),
        StructField("Lsa_summary", StringType(), True),
        StructField("Summarized", IntegerType(), True),
        StructField("Sentiment_score", IntegerType(), True),
        StructField("UUID", StringType(), True),
        StructField("Text", StringType(), True),
    ]
)

# Read the CSV file to Spark dataframe
df_summarized_news = spark.read.csv(
    FILE_PATH_SUMMARIZED_NEWS, header=True, schema=summarized_news_schema
)

# Store in memory
df_summarized_news.persist()

# Verify
print(f"Row count for df_summarized_news: {df_summarized_news.count()}")
df_summarized_news.show(5, truncate=True)

Row count for df_summarized_news: 5331
+-------------------+--------------------+------------+--------------------+---------+------+--------------------+--------------------+----------+---------------+--------------------+--------------------+
|               Date|       Article_title|Stock_symbol|                 Url|Publisher|Author|             Article|         Lsa_summary|Summarized|Sentiment_score|                UUID|                Text|
+-------------------+--------------------+------------+--------------------+---------+------+--------------------+--------------------+----------+---------------+--------------------+--------------------+
|2023-12-17 07:00:00|What Are the Best...|        AMZN|https://www.nasda...|     NULL|  NULL|It's nearly impos...|1 Wall Street Ana...|         1|              0|003d7667-7beb-452...|What Are the Best...|
|2023-12-17 07:00:00|Amazon (AMZN) Sto...|        AMZN|https://www.nasda...|     NULL|  NULL|Amazon (AMZN) clo...|Amazon (AMZN) Sto...|      

Retrieve the summaries and initialize the large-language model.

In [14]:
# Get the UUID, Date, Url, and Lsa_summary from the first row
collection = df_summarized_news.select(
    F.collect_list("UUID"),
    F.collect_list("Date"),
    F.collect_list("Lsa_summary"),
).first()
uuids = collection[0]
dates = collection[1]
lsa_summaries = collection[2]

# Verify
print(f"Number of uuids: {len(uuids)}")
print(f"Number of dates: {len(dates)}")
print(f"Number of lsa_summaries: {len(lsa_summaries)}")

# Initializing the OllamaLLM
try:
    llm = ChatOllama(
        model=MODEL, temperature=TEMPERATURE, num_predict=MAX_OUTPUT_TOKENS
    )
    print("ChatOllama instance created successfully!")
except Exception as e:
    print("Error creating ChatOllama instance:", e)

Number of uuids: 5331
Number of dates: 5331
Number of lsa_summaries: 5331
ChatOllama instance created successfully!


Feed the summmmaries to the model in batches and capture the resulting sentiment score.

In [11]:
# Holds the resulting output, i.e. the sentiment scores
sentiments = []

# Iterate in batches
for i in range(4445, len(lsa_summaries), BATCH_SIZE):
    batch_summaries = lsa_summaries[i : i + BATCH_SIZE]
    batch_uuids = uuids[i : i + BATCH_SIZE]

    num_text = len(batch_summaries)
    # print(f"#{i}, num_text: {num_text}")

    batch_text = " ".join(
        [
            f"### {STOCK_SYMBOL_UPPER} Stock News: {summary} "
            for summary in batch_summaries
        ]
    )
    # print(f"#{i}, batch_text: {batch_text}")

    chat_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                f"Forget all previous instructions. You are a financial expert with stock recommendation experience. Based on news for a specific stock, provide a sentiment score in the range of 1 to 5 inclusive, where 1 is negative, 2 is somewhat negative, 3 is neutral, 4 is somewhat positive, and 5 is positive. {num_text} summerized news will be passed in each time. You will provide {num_text} scores, one score for each of the summerized news in the format as shown below in the response from the assistant.",
            ),
            (
                "user",
                f"### AAPL Stock News: Apple (AAPL) increased 22%. ### AAPL Stock News: Apple (AAPL) price decreased 30%. ### MSFT Stock News: Microsoft (MSTF) price has not changed. ### AAPL Stock News: Apple (AAPL) announced the new iPhone 15. ### AAPL Stock News: Apple (AAPL) will release the Vison Pro on Feb 2, 2024.",
            ),
            ("ai", "5, 1, 3, 4, 4"),
            ("user", batch_text),
        ]
    )

    messages = chat_template.format_messages(num_text=num_text)
    # print(f"#{i}, messages: {messages}")

    response = llm.invoke(messages)
    print(f"#{i}: {response.content}")

    # Loop through each batch and append the sentiment scores to the list
    sentiments.extend(
        int(sentiment.strip()) for sentiment in response.content.split(",")
    )

NameError: name 'lsa_summaries' is not defined

In [ ]:
# Save the sentiments to a file
with open(FILE_PATH_SENTIMENT_SCORES, "w") as f:
    f.writelines(f"{sentiment}\n" for sentiment in sentiments)

In [16]:
# Verify
print(f"Number of lsa_summaries: {len(lsa_summaries)}")
print(f"Number of sentiments: {len(sentiments)}")
print(f"Row count for df_summarized_news: {df_summarized_news.count()}")

Number of lsa_summaries: 5331
Number of sentiments: 5331
Row count for df_summarized_news: 5331


In [15]:
# TO BE EXECUTED ONLY IF THE NUMBER OF SENTIMENTS DOES NOT MATCH THE NUMBER OF SUMMARIES
# Step 1: Re-run the sentiment score for the affected batch
# Step 2: Update the missing scores to the sentiments.csv file
# Step 3: Verify the number of sentiments in the sentiments.csv file matches the number of summaries
# Step 4: Execute code below to copy out the sentiments from sentiments.csv file to the sentiments list

# sentiment = []
# with open(FILE_PATH_SENTIMENT_SCORES, "r") as f:
#     sentiments = [int(sentiment.strip()) for sentiment in f if sentiment.strip()]
# print(f"Number of sentiments: {len(sentiments)}")

Number of sentiments: 5331


Save a copy of the sentiment scored news dataset.

In [17]:
if len(sentiments) != len(lsa_summaries):
    raise ValueError("The number of sentiments does not match the number of summaries.")

# Define the schema
scored_news_schema = StructType(
    [
        StructField("UUID", StringType(), True),
        StructField("Date", TimestampType(), True),
        StructField("Sentiment_score", IntegerType(), True),
        StructField("Lsa_summary", StringType(), True),
    ]
)

# Combine lists into a list of tuples
scored_news = list(zip(uuids, dates, sentiments, lsa_summaries))

# Create a dataframe with the specified schema
df_scored_news = spark.createDataFrame(scored_news, schema=scored_news_schema)

# Specify the directories
FOLDER_SCORED_NEWS = "stock_news/scored"
TEMP_FOLDER_SCORED_NEWS = "stock_news/scored/news_{STOCK_SYMBOL_LOWER}"

# Write to a single CSV file
df_scored_news.coalesce(1).write.csv(
    TEMP_FOLDER_SCORED_NEWS, sep=",", header=True, mode="overwrite"
)

# Move the part file to the desired filename
for filename in os.listdir(TEMP_FOLDER_SCORED_NEWS):
    if filename.startswith("part-"):
        shutil.move(
            os.path.join(TEMP_FOLDER_SCORED_NEWS, filename),
            os.path.join(FOLDER_SCORED_NEWS, FILE_NAME_NEWS),
        )

# Remove the temporary directory
shutil.rmtree(TEMP_FOLDER_SCORED_NEWS)

# Uncache the dataframe in a non-blocking operation to free up memory.
df_summarized_news.unpersist(blocking=False)

# Delete variables
del (
    summarized_news_schema,
    scored_news_schema,
    collection,
    uuids,
    dates,
    lsa_summaries,
    sentiments,
    llm,
    chat_template,
    messages,
    response,
    scored_news,
    df_summarized_news,
    df_scored_news,
)

24/10/24 20:17:09 WARN TaskSetManager: Stage 23 contains a task of very large size (1471 KiB). The maximum recommended task size is 1000 KiB.


DataFrame[Date: timestamp, Article_title: string, Stock_symbol: string, Url: string, Publisher: string, Author: string, Article: string, Lsa_summary: string, Summarized: int, Sentiment_score: int, UUID: string, Text: string]

# Exponentially Decay the Sentiment Score of the News

Read in the sentiment scored stock news.

In [18]:
# Define the schemas
scored_news_schema = StructType(
    [
        StructField("UUID", StringType(), True),
        StructField("Date", TimestampType(), True),
        StructField("Sentiment_score", IntegerType(), True),
        StructField("Lsa_summary", StringType(), True),
    ]
)

df_scored_news = spark.read.csv(
    FILE_PATH_SCORED_NEWS, header=True, schema=scored_news_schema
)

# Verify
print(f"Row count for df_scored_news: {df_scored_news.count()}")
df_scored_news.show(5, truncate=True)

Row count for df_scored_news: 5331
+--------------------+-------------------+---------------+--------------------+
|                UUID|               Date|Sentiment_score|         Lsa_summary|
+--------------------+-------------------+---------------+--------------------+
|003d7667-7beb-452...|2023-12-17 07:00:00|              2|1 Wall Street Ana...|
|2ffd76dd-00ef-44e...|2023-12-17 07:00:00|              1|Amazon (AMZN) Sto...|
|85448648-d4ed-45b...|2023-12-17 05:00:00|              3|3 Top Ranked Stoc...|
|064ff69c-6f18-42b...|2023-12-17 03:00:00|              4|TotalEnergies (TT...|
|290b0dd6-92bc-4bc...|2023-12-17 02:00:00|              2|US 'buy now, pay ...|
+--------------------+-------------------+---------------+--------------------+
only showing top 5 rows



Calculate the average sentiment scores for each date. Then, populate the missing dates.

In [19]:
# Extracting only the date component and sort by date
df_scored_news_copy = df_scored_news.select(
    "UUID", F.to_date("Date").alias("Date"), "Sentiment_score", "Lsa_summary"
).orderBy("Date")

# Group by "Date" and calculate the average "Sentiment_score"
df_avg_sentiment = df_scored_news_copy.groupBy("Date").agg(
    F.avg("Sentiment_score").alias("Sentiment_avg")
)

# Retrieve the start and end date
start_date = df_avg_sentiment.agg(F.min("Date")).collect()[0][0]
end_date = df_avg_sentiment.agg(F.max("Date")).collect()[0][0]

# Initialize a dataframe with the start and end dates
df_date_range = spark.createDataFrame(
    [(start_date, end_date)], ["start_date", "end_date"]
)

# Generate a date range using sequence
df_dates = df_date_range.select(
    F.expr("sequence(to_date(start_date), to_date(end_date), interval 1 day)").alias(
        "Date"
    )
)

# Explode the array to get individual dates
df_dates = df_dates.select(F.explode("Date").alias("Date"))

# Join the complete date range with the original dataframe
df_avg_sentiment_filled = df_dates.join(df_avg_sentiment, on="Date", how="left")

# Store in memory
df_avg_sentiment_filled.persist()

# Verify
print(f"Row count for df_avg_sentiment_filled: {df_avg_sentiment_filled.count()}")
df_avg_sentiment_filled.show(df_avg_sentiment_filled.count(), truncate=False)

Row count for df_avg_sentiment_filled: 1330
+----------+------------------+
|Date      |Sentiment_avg     |
+----------+------------------+
|2020-04-27|3.0833333333333335|
|2020-04-28|2.78125           |
|2020-04-29|2.966666666666667 |
|2020-04-30|3.2777777777777777|
|2020-05-01|3.548780487804878 |
|2020-05-02|3.0               |
|2020-05-03|2.0               |
|2020-05-04|3.0833333333333335|
|2020-05-05|3.375             |
|2020-05-06|3.2777777777777777|
|2020-05-07|3.5               |
|2020-05-08|3.5               |
|2020-05-09|2.0               |
|2020-05-10|4.5               |
|2020-05-11|2.8636363636363638|
|2020-05-12|3.2222222222222223|
|2020-05-13|3.388888888888889 |
|2020-05-14|3.0               |
|2020-05-15|3.4583333333333335|
|2020-05-16|3.0               |
|2020-05-17|2.0               |
|2020-05-18|3.5               |
|2020-05-19|2.6666666666666665|
|2020-05-20|3.3333333333333335|
|2020-05-21|3.1666666666666665|
|2020-05-22|3.6               |
|2020-05-23|3.0             

Apply an exponential decay algorithm to the missing average sentiment scores.

In [20]:
# Create a window specification to get the last valid sentiment and last valid date
window_spec = Window.orderBy("Date").rowsBetween(
    Window.unboundedPreceding, Window.currentRow
)

# Get last valid sentiment
df_avg_sentiment_filled = df_avg_sentiment_filled.withColumn(
    "Last_valid_sentiment", F.last("Sentiment_avg", ignorenulls=True).over(window_spec)
)

# Get last valid date only when sentiment is not null
df_avg_sentiment_filled = df_avg_sentiment_filled.withColumn(
    "Last_valid_date",
    F.last(
        F.when(F.col("Sentiment_avg").isNotNull(), F.col("Date")), ignorenulls=True
    ).over(window_spec),
)

# Calculate the number of days since the last valid sentiment
df_avg_sentiment_filled = df_avg_sentiment_filled.withColumn(
    "Days_since_last_valid", (F.datediff(F.col("Date"), F.col("Last_valid_date")))
)

# Calculate decayed sentiment for rows where the average sentiment is null
df_avg_sentiment_filled = df_avg_sentiment_filled.withColumn(
    "Decayed_sentiment",
    F.when(
        F.col("Sentiment_avg").isNull(),
        BASE_VALUE
        + (F.col("Last_valid_sentiment") - BASE_VALUE)
        * F.exp(-DECAY_RATE * F.col("Days_since_last_valid")),
    ).otherwise(F.col("Sentiment_avg")),
)

# Verify
print(f"Row count for df_avg_sentiment_filled: {df_avg_sentiment_filled.count()}")
df_avg_sentiment_filled.show(df_avg_sentiment_filled.count(), truncate=False)

Row count for df_avg_sentiment_filled: 1330


24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 2

+----------+------------------+--------------------+---------------+---------------------+------------------+
|Date      |Sentiment_avg     |Last_valid_sentiment|Last_valid_date|Days_since_last_valid|Decayed_sentiment |
+----------+------------------+--------------------+---------------+---------------------+------------------+
|2020-04-27|3.0833333333333335|3.0833333333333335  |2020-04-27     |0                    |3.0833333333333335|
|2020-04-28|2.78125           |2.78125             |2020-04-28     |0                    |2.78125           |
|2020-04-29|2.966666666666667 |2.966666666666667   |2020-04-29     |0                    |2.966666666666667 |
|2020-04-30|3.2777777777777777|3.2777777777777777  |2020-04-30     |0                    |3.2777777777777777|
|2020-05-01|3.548780487804878 |3.548780487804878   |2020-05-01     |0                    |3.548780487804878 |
|2020-05-02|3.0               |3.0                 |2020-05-02     |0                    |3.0               |
|2020-05-0

Save a copy of the dataset.

In [21]:
# Specify the directories
FOLDER_DECAYED_NEWS = "stock_news/decayed"
TEMP_FOLDER_DECAYED_NEWS = "stock_news/decayed/news_{STOCK_SYMBOL_LOWER}"

# Write to a single CSV file
df_avg_sentiment_filled.coalesce(1).write.csv(
    TEMP_FOLDER_DECAYED_NEWS, sep=",", header=True, mode="overwrite"
)

# Move the part file to the desired filename
for filename in os.listdir(TEMP_FOLDER_DECAYED_NEWS):
    if filename.startswith("part-"):
        shutil.move(
            os.path.join(TEMP_FOLDER_DECAYED_NEWS, filename),
            os.path.join(FOLDER_DECAYED_NEWS, FILE_NAME_NEWS),
        )

# Remove the temporary directory
shutil.rmtree(TEMP_FOLDER_DECAYED_NEWS)

# Uncache the dataframe in a non-blocking operation to free up memory.
df_avg_sentiment_filled.unpersist(blocking=False)

# Delete variables
del (
    window_spec,
    start_date,
    end_date,
    df_scored_news,
    df_scored_news_copy,
    df_avg_sentiment,
    df_date_range,
    df_dates,
    df_avg_sentiment_filled,
)

24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 20:17:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/24 2

# Integrate the News and Price Datasets

Ingest the decayed sentiment dataset and the price dataset.

In [3]:
# Define the schemas
decayed_sentiment_schema = StructType(
    [
        StructField("Date", DateType(), True),
        StructField("Sentiment_avg", DecimalType(18, 16), True),
        StructField("Last_valid_sentiment", DecimalType(18, 16), True),
        StructField("Last_valid_date", DateType(), True),
        StructField("Days_since_last_valid", IntegerType(), True),
        StructField("Decayed_sentiment", DecimalType(18, 16), True),
    ]
)
price_schema = StructType(
    [
        StructField("Date", DateType(), True),
        StructField("Open", DecimalType(24, 16), True),
        StructField("High", DecimalType(24, 16), True),
        StructField("Low", DecimalType(24, 16), True),
        StructField("Close", DecimalType(24, 16), True),
        StructField("Adj_close", DecimalType(24, 16), True),
        StructField("Volume", LongType(), True),
    ]
)

# Read price data from CSV files to Spark dataframes
df_decayed_sentiment = spark.read.csv(
    FILE_PATH_DECAYED_NEWS, header=True, schema=decayed_sentiment_schema
)
df_price = spark.read.csv(
    FILE_PATH_PREPROCESSED_PRICE, header=True, schema=price_schema
)

# Verify
print(f"Row count for df_decayed_sentiment: {df_decayed_sentiment.count()}")
df_decayed_sentiment.show(5, truncate=True)
print(f"Row count for df_price: {df_price.count()}")
df_price.show(5, truncate=True)

Row count for df_decayed_sentiment: 5242
+----------+------------------+--------------------+---------------+---------------------+------------------+
|      Date|     Sentiment_avg|Last_valid_sentiment|Last_valid_date|Days_since_last_valid| Decayed_sentiment|
+----------+------------------+--------------------+---------------+---------------------+------------------+
|2009-08-10|3.1666666666666665|  3.1666666666666665|     2009-08-10|                    0|3.1666666666666665|
|2009-08-11|              NULL|  3.1666666666666665|     2009-08-10|                    1|3.1010884432854390|
|2009-08-12|              NULL|  3.1666666666666665|     2009-08-10|                    2|3.0613132401952403|
|2009-08-13|              NULL|  3.1666666666666665|     2009-08-10|                    3|3.0371883600247385|
|2009-08-14|              NULL|  3.1666666666666665|     2009-08-10|                    4|3.0225558805394352|
+----------+------------------+--------------------+---------------+-----------

Merge the price dataset with the average sentiment score taken from news dataset.

In [4]:
# Drop unused fields
df_decayed_sentiment = df_decayed_sentiment.drop(
    "Sentiment_avg", "Last_valid_sentiment", "Last_valid_date", "Days_since_last_valid"
)

# Join the price data with the average sentiment data
df_combined = df_price.join(df_decayed_sentiment, on="Date", how="left")

# Store in memory
df_combined.persist()

# Convert the decayed sentiment values to 3 if they are null
df_combined = df_combined.withColumn(
    "Decayed_sentiment",
    F.when(
        F.col("Decayed_sentiment").isNull() | (F.col("Decayed_sentiment") == ""),
        BASE_VALUE,
    ).otherwise(F.col("Decayed_sentiment")),
)

# Verify
df_combined.show(20, truncate=False)

+----------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+------------------+
|Date      |Open               |High               |Low                |Close              |Adj_close          |Volume  |Decayed_sentiment |
+----------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+------------------+
|2023-12-28|34.6800003051757800|34.8120002746582000|34.1980018615722660|34.5499992370605500|34.5499992370605500|3900000 |3.0000000000000000|
|2023-12-27|34.2299995422363300|35.0349998474121100|33.7500000000000000|34.8100013732910160|34.8100013732910160|5798100 |3.0000000000000000|
|2023-12-26|33.8600006103515600|34.1199989318847660|33.5000000000000000|33.8699989318847660|33.8699989318847660|4505900 |3.0000000000000000|
|2023-12-22|32.9599990844726560|34.4399986267089840|32.8800010681152340|33.7700004577636700|33.7700004577636700|9064600 |3.0000000000000000|
|2023-12-21|3

Normalize the sentiments to between 0 and 1.

In [5]:
# Normalize the Decayed_sentiment column
df_combined = df_combined.withColumn(
    "Normalized_sentiment",
    (F.col("Decayed_sentiment") - MIN_VALUE) / (MAX_VALUE - MIN_VALUE),
)

# Verify
df_combined.show(20, truncate=False)

+----------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+------------------+--------------------+
|Date      |Open               |High               |Low                |Close              |Adj_close          |Volume  |Decayed_sentiment |Normalized_sentiment|
+----------+-------------------+-------------------+-------------------+-------------------+-------------------+--------+------------------+--------------------+
|2023-12-28|34.6800003051757800|34.8120002746582000|34.1980018615722660|34.5499992370605500|34.5499992370605500|3900000 |3.0000000000000000|0.500000000000000000|
|2023-12-27|34.2299995422363300|35.0349998474121100|33.7500000000000000|34.8100013732910160|34.8100013732910160|5798100 |3.0000000000000000|0.500000000000000000|
|2023-12-26|33.8600006103515600|34.1199989318847660|33.5000000000000000|33.8699989318847660|33.8699989318847660|4505900 |3.0000000000000000|0.500000000000000000|
|2023-12-22|32.9599990844726

Save a copy of the dataset.

In [6]:
# Specify the directories
FOLDER_COMBINED = "stock_combined"
TEMP_FOLDER_COMBINED = "stock_combined/{STOCK_SYMBOL_LOWER}"

# Write to a single CSV file
df_combined.coalesce(1).write.csv(
    TEMP_FOLDER_COMBINED, sep=",", header=True, mode="overwrite"
)

# Move the part file to the desired filename
for filename in os.listdir(TEMP_FOLDER_COMBINED):
    if filename.startswith("part-"):
        shutil.move(
            os.path.join(TEMP_FOLDER_COMBINED, filename),
            os.path.join(FOLDER_COMBINED, FILE_NAME_COMBINED),
        )

# Remove the temporary directory
shutil.rmtree(TEMP_FOLDER_COMBINED)

# Uncache the dataframe in a non-blocking operation to free up memory.
df_combined.unpersist(blocking=False)

# Delete variables
del decayed_sentiment_schema, price_schema, df_price, df_decayed_sentiment, df_combined